In [3]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

In [4]:
default_config = {
    "semester": 1,
    "local_search": {
        "algorithm": "simulated_annealing",
        "config": {
            "neighborhood": {
                "algorithm": "random_swap",
                "random_swap": {
                    "neighborhood_size": 25
                },
                "random_range_swap": {
                    "neighborhood_size_factor": 0.1,
                    "range_size_factor": 0.1
                },
                "distance_swap": {
                    "distance_percentage": 0.1
                },
                "swap": False
            },
            "simulated_annealing": {
                "initial_temperature": 100,
                "cooling_rate": 0.1,
                "max_iteration": 250,
                "max_time": 30,
                "max_iteration_without_improvement": 50,
            },
            "tabu_search": {
                "tabu_list_size": 50,
                "max_iteration": 1000,
                "max_time": 60,
                "max_iteration_without_improvement": 100,
                "max_time_without_improvement": 5
            }
        }
    },
    "algorithm": {
        "algorithm": "genetic_algorithm",
        "config": {
            "max_iteration": 1000,
            "population_size": 25,
            "elitism_size": 2,
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 1
                },
                "assistant_distribution": {
                    "max_group_threshold": 15,
                    "max_shift_threshold": 6,
                    "group_penalty": 0.25,
                    "shift_penalty": 1
                },
                "timeslot_conflict": {
                    "assistant_conflict_penalty": 2,
                    "group_conflict_penalty": 1
                },
            },
            "operator": {
                "selection": {
                    "roulette_wheel": True,
                    "tournament": True,
                    "elitism": False,
                    "tournament_size": 10
                },
                "crossover": {
                    "single_point": False,
                    "two_point": False,
                    "uniform": True,
                    "crossover_probability": 0.1,
                    "uniform_probability": 0.5
                },
                "mutation": {
                    "swap": True,
                    "shift": False,
                    "random": True,
                    "mutation_probability": 0.05
                },
                "repair": {
                    "time_slot": True
                }
            }
        }
    }
}




In [5]:
from scheduling_algorithm.data_parser import ModuleData
from scheduling_algorithm.algorithms import (
    GeneticAlgorithm,
    GeneticLocalSearch
)
from scheduling_algorithm.utils.solution_generator import SolutionGenerator
from scheduling_algorithm.factory import WeeklyFactory

In [6]:
modules = ModuleData.get_modules_by_semester(1)

In [7]:
# generator = SolutionGenerator.from_data(default_config)
weekly_generator = SolutionGenerator.from_data(default_config)

Loading configuration...
Configuration loaded successfully.
Creating FitnessManager with fitness functions: 
Fitness(name=GroupAssignmentCapacityFitness, max_threshold=3, conflict_penalty=1)
Fitness(name=AssistantDistributionFitness, max_group_threshold=15, max_shift_threshold=6, group_penalty=0.25, shift_penalty=1)
Fitness(name=TimeslotConflictFitness, assistant_conflict_penalty=2, group_conflict_penalty=1)
Configured selection functions:  [Selection(name=RouletteWheelSelection), Selection(name=TournamentSelection)]
Configuring crossover operator:  [Crossover(name=UniformCrossover)]
Configuring mutation operator:  [Mutation(name=SwapMutation), Mutation(name=RandomMutation)]
Configuring repair operator:  [Repair(name=RepairTimeSlot)]
Creating Genetic Algorithm Object from Configuration File
Population Size:  25
Max Iteration:  1000


In [8]:
import cProfile as profile
import pstats
from pstats import SortKey

profiler = profile.Profile()
profiler.enable()
solution = weekly_generator.generate_solution_weekly_test()
profiler.disable()

Generating population for module 1 week 1
Generating population for module 1 week 2
Generating population for module 1 week 3
Generating population for module 1 week 4
Generating population for module 1 week 5
Module 1 week 5 has no population, all the remaining chapter are already assigned on previous weeks
Starting the schedule generation algorithm...


In [14]:
solution[0]

{'laboratory': 1,
 'module': 1,
 'chapter': 1,
 'group': 1,
 'assistant': 5,
 'time_slot': TimeSlot(date=1663054200.53, day='Tuesday', shift='Shift1')}

In [9]:
stats = pstats.Stats(profiler).sort_stats(SortKey.TIME)
stats.print_stats()

         57958191 function calls (57828088 primitive calls) in 83.140 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   104100   41.664    0.000   54.901    0.001 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\manager.py:30(__call__)
    96000   19.738    0.000   19.857    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\operator\repair\time_slot_repair.py:22(__call__)
   192130    3.349    0.000    3.428    0.000 {built-in method numpy.array}
  9895212    2.784    0.000    2.784    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\manager.py:25(<lambda>)
 14990400    2.290    0.000    2.290    0.000 {method 'add' of 'set' objects}
   104100    1.494    0.000    1.824    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitnes

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'dict'